# NLP_2 Tokenization, Stemming, and Lemmatization with SpaCy Library

[Python for NLP: Tokenization, Stemming, and Lemmatization with SpaCy Library](https://stackabuse.com/python-for-nlp-tokenization-stemming-and-lemmatization-with-spacy-library/)

## Introduction to SpaCy

SpaCy 和 NLTK 都是進行自然語言處理相當熱門的套件，這兩者的差異在於 NLTK 提供了許多不同的演算法，讓分析者可以自行建構模型，並且透過不同的方式去解決問題，而 SpaCy 提供的是單一能夠解決問題的最佳演算方式。本文件主要會利用 SpaCy 套件來進行 NLP 中幾個核心的操作，包括 tokenization、stemming 和 lemmatization。

### Basic Functionality

在安裝完畢 SpaCy 的套件後，接下來必須安裝語言模型，用以進行後續的操作，這邊以英文語言模型作為示範。安裝完畢後，就可以呼叫套件和安裝好的模型，並將模型儲存成物件。

In [ ]:
$ python -m spacy download en # 安裝模型

In [2]:
import spacy
sp = spacy.load('en') # 把英文模型存成 sp 物件囉

語言模型的物件可以針對句子和文章自動切分成數個 tokens，所謂 token 就是句子中具有語言涵義的單一個體。舉例來說，把 Manchester United is looking to sign a forward for $90 million 餵到模型之中，就可以利用迴圈找出該句子包含的所有 tokens，以及這些 tokens 的性質（文字```text```、文法特性```pos_```、相依關係```dep_```等）。

In [22]:
sentence = sp('Manchester United is looking to sign a forward for $90 million')

for word in sentence:
    print(word.text) # 找出所有 tokens

Manchester
United
is
looking
to
sign
a
forward
for
$
90
million


In [24]:
for word in sentence:
    print(word.text,  word.pos_) # 找出所有 tokens 以及其文法特性

Manchester PROPN
United PROPN
is AUX
looking VERB
to PART
sign VERB
a DET
forward NOUN
for ADP
$ SYM
90 NUM
million NUM


In [25]:
sentence2 = sp(u"Manchester United isn't looking to sign any forward.")

for word in sentence2:
    print(word.text,  word.pos_, word.dep_) # 找出所有 tokens、文法特性及相依關係

Manchester PROPN compound
United PROPN nsubj
is AUX aux
n't PART neg
looking VERB ROOT
to PART aux
sign VERB xcomp
any DET advmod
forward ADV advmod
. PUNCT punct


除了句子以外，SpaCy 物件也能夠處理文章，並且透過迴圈搭配```sents```把每個句子找出來。此外也能夠過 index 找出特定位置存在的 token 為何，並利用```is_sent_start```確認該 token 是否為句首。

In [51]:
document = sp('Hello from Stackabuse. The site with the best Python Tutorials. What are you looking for?')
for sentence in document.sents:
    print(sentence)

Hello from Stackabuse.
The site with the best Python Tutorials.
What are you looking for?


In [27]:
document[4]

The

In [28]:
document[4].is_sent_start

True

## Tokenization

如同先前所呈現的，tokenization 就是將文章或句子切分成數個含有語意的單位，也就是 token。

以下再舉一個例子，對 "They're leaving U.K. for U.S.A." 這句話進行 tokenization，會發現前後引號也被視為 token 找出來了，不過電腦卻正確判斷出 U.K. 和 U.S.A. 的 . 為縮寫而非句號。

而對 Hello, I am non-vegetarian, email me the menu at abc-xyz@gmai.com 進行 tokenization，可以發現郵件地址成功被指認出來，然而 non-vegetarian 的 - 被視為 token 獨立出來了。

In [53]:
sentence3 = sp('"They\'re leaving U.K. for U.S.A."')

for word in sentence3:
    print(word.text)

"
They
're
leaving
U.K.
for
U.S.A.
"


In [33]:
sentence4 = sp("Hello, I am non-vegetarian, email me the menu at abc-xyz@gmai.com")

for word in sentence4:
    print(word.text)

Hello
,
I
am
non
-
vegetarian
,
email
me
the
menu
at
abc-xyz@gmai.com


In [34]:
len(sentence4) # 對 tokenize 過後的物件取 len() 可以直接計算 tokens 的數量喔！

14

### Detecting Entities

除了找出一個句子的所有 tokens 之外，有時候也需要找出句子當中的 entities，也就是組織、地方、公司或者人名等等。之所以要特別提出 entity 的概念，是因為在一般 tokenize 的過程中，會忽略掉部分多字複合成的單一語意詞。舉例來說，針對 Manchester United is looking to sign Harry Kane for \$90 million 進行 tokenize，Manchester United 這個詞就會被拆成兩個 tokens，Harry Kane 和 $90 million 也是。

這種找出句子中特別的 entity 的過程，就是在 NLP 中所謂的 NER（Named Entity Recognition，命名實體辨別），要找出句子當中的 entities，只需要從 SpaCy 物件中找出```ents```性質即可，針對找出來的不同 entities，還能取出各自的文字```text```和類型標籤```label_```。

In [36]:
sentence5 = sp('Manchester United is looking to sign Harry Kane for $90 million')

for word in sentence5:
    print(word.text)

Manchester
United
is
looking
to
sign
Harry
Kane
for
$
90
million


In [39]:
for entity in sentence5.ents: # 對 SpaCy 的 entities 進行迴圈
    print(entity.text + ' - ' + entity.label_ + ' - ' + str(spacy.explain(entity.label_))) # 取出文字和標籤

Manchester United - PERSON - People, including fictional
Harry Kane - PERSON - People, including fictional
$90 million - MONEY - Monetary values, including unit


### Detecting Nouns

除了找出 tokens 和 entities 之外，SpaCy 套件還能夠直接找出句子當中的名詞，只要利用```noun_chunks```性質即可。

In [3]:
sentence6 = sp('Latest Rumours: Manchester United is looking to sign Harry Kane for $90 million') 

for noun in sentence6.noun_chunks:
    print(noun.text)

Manchester United
Harry Kane


## Stemming

所謂 stemming 就是詞幹提取，也就是把找出一個單字的最原始型態，在英文、德文等語言當中，經常會碰到許多不同的源自同一基礎的字，例如 compute、computer、computing、computed 等等，從分析的角度來看，大多會希望這些字詞能夠被視為同一單詞。

SpaCy 並沒有可以進行 stemming 的函數，因次要使用到 NLTK 套件，分別使用 Porter Stemmer 和 Snowball Stemmer。 

### Porter Stemmer

In [41]:
import nltk

from nltk.stem.porter import *

In [42]:
stemmer = PorterStemmer() # 產生進行 Porter Stemmer 的物件

In [43]:
tokens = ['compute', 'computer', 'computed', 'computing']

In [44]:
for token in tokens:
    print(token + ' --> ' + stemmer.stem(token)) # 取出每個 tokens 的 root form

compute --> comput
computer --> comput
computed --> comput
computing --> comput


### Snowball Stemmer

In [46]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer(language='english') # 產生進行 Snowball Stemmer 的物件

tokens = ['compute', 'computer', 'computed', 'computing']

for token in tokens:
    print(token + ' --> ' + stemmer.stem(token)) # 取出每個 tokens 的 root form

compute --> comput
computer --> comput
computed --> comput
computing --> comput


根據前兩個 stemming 的結果，發現都找出了 comput 作為詞幹，然而這個卻是一個不存在的單字，因此一般會使用另一個叫做 lemmatization 的方法，也就是所謂的詞形還原，把先前提到的那些同源詞，還原到一個真實出現在字典的單詞。

### Lemmatization

Lemmatization 的操作使用到 SpaCy 套件，對句子當中的 tokens 取出```lemma_```性質即可找出其 root form。

In [47]:
sentence7 = sp('compute computer computed computing')

In [49]:
for word in sentence7:
    print(word.text,  word.lemma_) # 取出 tokens 的詞和還原詞形。

compute compute
computer computer
computed compute
computing computing


In [50]:
sentence8 = sp(u'A letter has been written, asking him to be released')

for word in sentence8:
    print(word.text + '  ===>', word.lemma_)

A  ===> a
letter  ===> letter
has  ===> have
been  ===> be
written  ===> write
,  ===> ,
asking  ===> ask
him  ===> -PRON-
to  ===> to
be  ===> be
released  ===> release
